### Dreambooth 模型微调
DreamBooth 是一种深度学习生成模型，用于微调现有的文本到图像模型，由 Google Research 和波士顿大学的研究人员于 2022 年开发。最初使用 Google 自己的 Imagen 文本到图像模型开发，DreamBooth 的实现可以应用到其他文本到图像模型，它可以让模型通过的三到五张图像对一个主题进行训练后生成更精细和个性化的输出。

![](../../images/dreambooth.png)

接下来我们将使用 DreamBooth 来微调我们的 stable diffusion 模型.

#### Notebook 步骤
1. 导入 boto3, sagemaker python SDK
2. 构建 dreambooth fine-tuning 镜像
3. 实现模型微调
   * 配置超参
   * 创建训练任务
4. 测试

#### 1. 导入 boto3, sagemaker python SDK

In [ ]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

images_s3uri = 's3://{0}/dreambooth/images/'.format(bucket)
models_s3uri = 's3://{0}/stable-diffusion/models/'.format(bucket)
dreambooth_s3uri = 's3://{0}/stable-diffusion/dreambooth/'.format(bucket)

#### 2. 构建 dreambooth fine-tuning 镜像
  如果你使用较小的实例，如 ml.t3.xlarge，从头构建需要 60~90 分钟的时间,为此workshop 提供了预构建docker image, 具体请参考Docker.public-ecr,修改代码train_dreambooth.py后需要重新运行

In [ ]:
!./build_push.sh

#### 3. 模型微调

   * image_uri: ecr仓库中的 docker 镜像地址
   * instance_type: 用于训练任务的实例大小 , 建议使用 ml.g4dn.xlarge, ml.g5.xlarge
   * class_prompt: 提示词类别
   * instance_prompt: 用于你的图片的关键词
   * model_name: 预训练的模型名称
   

In [ ]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}




image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sd-dreambooth-finetuning-v2'
#instance_type = 'ml.g4dn.2xlarge'
instance_type = 'local'

instance_prompt="photo\ of\ zwx\  man"
class_prompt="photo\ of\ a\ man"
s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'dreambooth', 'trained_models')
model_name="runwayml/stable-diffusion-v1-5"
instance_dir="/opt/ml/input/data/images/"
class_dir="/opt/ml/input/data/class_images/"



environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'model_name':'aws-trained-dreambooth-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': model_name, 
                    'instance_data_dir':instance_dir,
                    'class_data_dir':class_dir,
                    'with_prior_preservation':True,
                    'models_path': '/opt/ml/model/',
                    'instance_prompt': instance_prompt, 
                    'class_prompt':class_prompt,
                    'resolution':512,
                    'train_batch_size':1,
                    'sample_batch_size': 1,
                    'gradient_accumulation_steps':1,
                    'learning_rate':2e-06,
                    'lr_scheduler':'constant',
                    'lr_warmup_steps':0,
                    'num_class_images':50,
                    'max_train_steps':300,
                    'save_steps':100,
                    'attention':'xformers',
                    'prior_loss_weight': 0.5,
                    'use_ema':True,
                    'train_text_encoder':False,
                    'not_cache_latents':True,
                    'gradient_checkpointing':True,
                    'save_use_epochs': False,
                    'use_8bit_adam': False
}

hyperparameters = json_encode_hyperparameters(hyperparameters)



   * 创建训练任务

In [ ]:
from sagemaker.estimator import Estimator
inputs = {
    'images': f"s3://{bucket}/dreambooth/images/"
}


estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment
)
estimator.fit(inputs)

In [ ]:
dreambooth_model_data = estimator.model_data
print("Model artifact saved at:\n", dreambooth_model_data)

#### 4. 测试
  现在你可以使用推理笔记本加载您训练的模型

In [ ]:
#回到4.1 部署Stable Diffusion 模型 ,将andite/anything-v4.0替换为你fine tuning后的模型
framework_version = '1.10'
py_version = 'py38'

model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
    #'model_name':'andite/anything-v4.0',
    'model_name':'s3://<youbucket>/<model floder>',
    's3_bucket':bucket
}